In [2]:
import numpy as np
import vedo
import os
from tqdm import tqdm
from MeshObject import *
import pandas as pd
from Pipeline import *
from collections import defaultdict
import pymeshlab

In [3]:
# df = pd.read_pickle("original_data_statistics.pickle")
# class_type = "Knife"
# df = df[df["class"] == class_type]

# shape_path_orig = "../ShapeDatabase_INFOMR_orig"
# # pipeline = Pipeline(pipeline_parameters={"subdivide": {"threshold": 5610, "subdivision_type": "adaptive", "decimation_type": "simple"}})

# new_path_shape = "../ShapeDatabase_INFOMR"
# nontwomanifold_faces = defaultdict(list)
# # class_type, obj_name = "", ""
# obj_name = ""
# iterator = tqdm(zip(df["class"], df["name"]), total=len(df), desc=f"{class_type} - {obj_name}")
# ok = []
# for class_type, obj_name in iterator:
#     iterator.set_description(f"{class_type} - {obj_name}")
#     os.makedirs(os.path.join(new_path_shape, class_type), exist_ok=True)
#     path = os.path.join(shape_path_orig, class_type, obj_name)
#     new_path = os.path.join(new_path_shape, class_type, obj_name)
#     mesh = MeshObject(path, True, class_type=class_type, name=obj_name)
#     if not mesh.is_manifold():
#         nontwomanifold_faces[class_type] += [obj_name]
#     mesh: MeshObject = pipeline.normalize_shape(mesh)

#     # vedo.save(mesh, new_path)

In [4]:
# def sanitize_nonmanifoldness(pymesh_set):
#     operations_nonmanifoldness = [
#         partial(pymesh_set.meshing_repair_non_manifold_edges, method=1),
#         pymesh_set.compute_selection_by_non_manifold_edges_per_face,
#         pymesh_set.meshing_remove_selected_faces,
#         pymesh_set.meshing_repair_non_manifold_vertices,
#     ]

#     is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#     while not is_manifold:
#         for operation in operations_nonmanifoldness:
#             operation()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]

#     return pymesh_set

# def sanitize_geometry(pymesh_set):
#     def remove_disconnected_components(pymesh_set, nbfaceratio=0.25, ratio_selected_faces = 0.24): # elbow method?
#         pymesh_set.apply_filter("set_selection_all")
#         all_faces_num = pymesh_set.current_mesh().selected_face_number()
#         pymesh_set.set_selection_none()
#         pymesh_set.compute_selection_by_small_disconnected_components_per_face(nbfaceratio=nbfaceratio)
#         if pymesh_set.current_mesh().selected_face_number()/all_faces_num > ratio_selected_faces:
#             # print("??")
#             pymesh_set.meshing_remove_selected_faces()
#         pymesh_set.set_selection_none()

#         return pymesh_set

#     operations_cleaning = [
#         pymesh_set.meshing_remove_duplicate_faces,
#         partial(remove_disconnected_components, pymesh_set, nbfaceratio=0.25),
#         pymesh_set.meshing_merge_close_vertices,
#     ]

#     for operation in operations_cleaning:
#         operation()

#     return pymesh_set

# def sanitize_mesh(pymesh_set):       
#         pymesh_set = sanitize_geometry(pymesh_set)
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)

#         # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

#         pymesh_set.meshing_close_holes(refinehole=True, refineholeedgelen=pymeshlab.PercentageValue(1), maxholesize=75)
#         pymesh_set.set_selection_none()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)
       
#         return pymesh_set
    
# def resample_shape_pymeshlab(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
#     pymesh = vedo.utils.vedo2meshlab(vedo_mesh)
#     pymesh_set = pymeshlab.MeshSet()
#     pymesh_set.add_mesh(pymesh)
#     vedo_mesh = sanitize_mesh(pymesh_set)

#     subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_ls3_loop", loopweight=2, iterations=1)
#     if pymesh_set.current_mesh().vertex_number() < 100: # low vertex shapes' topology is heavily modified by any loop-adjacent subdivision algo
#         subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_midpoint", iterations=1)

#     print(pymesh_set.current_mesh().vertex_number())
#     last_vertex_count = -1
#     while pymesh_set.current_mesh().vertex_number() < threshold:
#         subdiv_method()
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#     last_vertex_count = -1
#     prev_meshset = pymeshlab.MeshSet()
#     prev_meshset.add_mesh(pymesh_set.current_mesh())
#     pymesh_set.apply_filter("set_selection_all")
#     current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
#     pymesh_set.set_selection_none()
#     while pymesh_set.current_mesh().vertex_number() > threshold:
#         pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.99, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
#         pymesh_set.apply_filter("set_selection_all")
#         modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
#         pymesh_set.set_selection_none()
#         print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
#         if modified_vertex_num/current_vertex_num < 0.8:
#             pymesh_set = prev_meshset
#             break
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#     print(pymesh_set.current_mesh().vertex_number(), "last!!")
#     print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

#     return MeshObject(pymesh_set.current_mesh(), visualize=True).compute_normals()

# # mesh = MeshObject("../ShapeDatabase_INFOMR_orig" + "/Knife/D01187.obj", True, class_type="House", name="m461")
# # mesh = MeshObject("../ShapeDatabase_INFOMR_orig" + "/Door/D01121.obj", True, class_type="House", name="m461")
# # mesh = MeshObject("../ShapeDatabase_INFOMR_orig" + "/Cellphone/D00979.obj", True, class_type="House", name="m461")
# # mesh = MeshObject("../ShapeDatabase_INFOMR_orig" + "/Monoplane/D00098.obj", True, class_type="House", name="m461")
# mesh = MeshObject("../ShapeDatabase_INFOMR_orig" + "/Knife/D01051.obj", True, class_type="House", name="m461")
# resample_shape_pymeshlab(mesh).show()

# # midpoint for really small vertex shapes? (<100)
# # loop for others?
# # loop(C2 continuity) = approximation
# # problem with sharp shapes - mention in report
# # butterfly(C1 continuity) = interpolation
# # middege(C1) = both

# # rm duplicate faces -> remove nm faces, split nm vertices -> ls3 with enhanced continuity (read paper and cite!!!) -> quadric decimation with quality=0.5, and all true args -> rm nm edges, vertices [problems with hole filling]
# # rm duplicate faces -> rm small components (ratio=0.25) -> merge vertices -> split nm edges -> rm nm edges -> split nm vertices -> close holes (r=75) -> ls3 subd with enhanced continuity -> decimation (with existing paras) -> works ok?

In [5]:
# def sanitize_nonmanifoldness(pymesh_set):
#     operations_nonmanifoldness = [
#         partial(pymesh_set.meshing_repair_non_manifold_edges, method=1),
#         pymesh_set.compute_selection_by_non_manifold_edges_per_face,
#         pymesh_set.meshing_remove_selected_faces,
#         pymesh_set.meshing_repair_non_manifold_vertices,
#     ]

#     is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#     while not is_manifold:
#         for operation in operations_nonmanifoldness:
#             operation()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]

#     return pymesh_set

# def sanitize_geometry(pymesh_set):
#     def remove_disconnected_components(pymesh_set, nbfaceratio=0.25, ratio_selected_faces = 0.24): # elbow method?
#         pymesh_set.apply_filter("set_selection_all")
#         all_faces_num = pymesh_set.current_mesh().selected_face_number()
#         pymesh_set.set_selection_none()
#         pymesh_set.compute_selection_by_small_disconnected_components_per_face(nbfaceratio=nbfaceratio)
#         r = pymesh_set.current_mesh().selected_face_number()/all_faces_num
#         if pymesh_set.current_mesh().selected_face_number()/all_faces_num > ratio_selected_faces: # do it per class?
#             # print("??")
#             pymesh_set.meshing_remove_selected_faces()
#         pymesh_set.set_selection_none()

#         return pymesh_set, r

#     operations_cleaning = [
#         pymesh_set.meshing_remove_duplicate_faces,
#         partial(remove_disconnected_components, pymesh_set, nbfaceratio=0.25),
#         pymesh_set.meshing_merge_close_vertices,
#     ]

#     r = None
#     for operation in operations_cleaning:
#         if isinstance(operation, partial):
#             _, r = operation()
#         else:
#             operation()

#     return pymesh_set, r

# def sanitize_mesh(pymesh_set):       
#         pymesh_set, r = sanitize_geometry(pymesh_set)
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)

#         # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

#         pymesh_set.meshing_close_holes(refinehole=True, refineholeedgelen=pymeshlab.PercentageValue(1), maxholesize=75)
#         pymesh_set.set_selection_none()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)
       
#         return pymesh_set, r
    
# def resample_shape_pymeshlab(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
#     pymesh = vedo.utils.vedo2meshlab(vedo_mesh)
#     pymesh_set = pymeshlab.MeshSet()
#     pymesh_set.add_mesh(pymesh)
#     vedo_mesh, r = sanitize_mesh(pymesh_set)

#     # print(pymesh_set.current_mesh().vertex_number())

#     subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_ls3_loop", loopweight=2, iterations=1)
#     if pymesh_set.current_mesh().vertex_number() < 100: # low vertex shapes' topology is heavily modified by any loop-adjacent subdivision algo
#         subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_midpoint", iterations=1)

#     last_vertex_count = -1
#     while pymesh_set.current_mesh().vertex_number() < threshold:
#         subdiv_method()
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#     last_vertex_count = -1
#     prev_meshset = pymeshlab.MeshSet()
#     prev_meshset.add_mesh(pymesh_set.current_mesh())
#     pymesh_set.apply_filter("set_selection_all")
#     current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
#     pymesh_set.set_selection_none()
#     while pymesh_set.current_mesh().vertex_number() > threshold:
#         pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.99, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
#         pymesh_set.apply_filter("set_selection_all")
#         modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
#         pymesh_set.set_selection_none()
#         # print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
#         if modified_vertex_num/current_vertex_num < 0.8:
#             pymesh_set = prev_meshset
#             break
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#     # print(pymesh_set.current_mesh().vertex_number(), "last!!")
#     # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

#     return MeshObject(pymesh_set.current_mesh(), visualize=True), r

# df = pd.read_pickle("original_data_statistics.pickle")
# class_type = "Knife"
# df = df[df["class"] == class_type]

# shape_path_orig = "../ShapeDatabase_INFOMR_orig"
# obj_name = ""
# iterator = tqdm(zip(df["class"], df["name"]), total=len(df), desc=f"{class_type} - {obj_name}")
# ok = []
# ratios = defaultdict(list)
# for class_type, obj_name in iterator:
#     iterator.set_description(f"{class_type} - {obj_name}")
#     path = os.path.join(shape_path_orig, class_type, obj_name)
#     mesh = MeshObject(path, True, class_type=class_type, name=obj_name)
#     mesh, ratio = resample_shape_pymeshlab(mesh)
#     ratios[class_type] += [ratio]

In [5]:
def sanitize_nonmanifoldness(pymesh_set):
    operations_nonmanifoldness = [
        partial(pymesh_set.meshing_repair_non_manifold_edges, method=0),
        # pymesh_set.compute_selection_by_non_manifold_edges_per_face,
        # pymesh_set.meshing_remove_selected_faces,
        pymesh_set.meshing_repair_non_manifold_vertices,
    ]

    is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
    while not is_manifold:
        for operation in operations_nonmanifoldness:
            operation()
            # print("?")
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        # print("done for loop")

    # print("done manifold")
    return pymesh_set

def sanitize_geometry(pymesh_set):
    def remove_disconnected_components(pymesh_set, nbfaceratio=0.05, ratio_selected_faces = 0.24): # elbow method?
        pymesh_set.apply_filter("set_selection_all")
        all_faces_num = pymesh_set.current_mesh().selected_face_number()
        pymesh_set.set_selection_none()
        pymesh_set.compute_selection_by_small_disconnected_components_per_face(nbfaceratio=nbfaceratio)
        r = pymesh_set.current_mesh().selected_face_number()/all_faces_num
        if r > ratio_selected_faces: # do it per class?
            # print("removed components")
            pymesh_set.meshing_remove_selected_faces()
        pymesh_set.set_selection_none()

        return pymesh_set, r

    operations_cleaning = [
        pymesh_set.meshing_remove_duplicate_faces,
        partial(remove_disconnected_components, pymesh_set, nbfaceratio=0.05),
        pymesh_set.meshing_merge_close_vertices,
    ]

    r = None
    for operation in operations_cleaning:
        if isinstance(operation, partial):
            _, r = operation()
        else:
            # print(operation.__name__)
            operation()

    return pymesh_set, r

def sanitize_mesh(pymesh_set):       
        pymesh_set, r = sanitize_geometry(pymesh_set)
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        if not is_manifold:
            pymesh_set = sanitize_nonmanifoldness(pymesh_set)

        # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])
        # print("pre_hole")

        pymesh_set.meshing_close_holes(refinehole=True, refineholeedgelen=pymeshlab.PercentageValue(1), maxholesize=75)
        # print("hole")
        pymesh_set.set_selection_none()
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        if not is_manifold:
            pymesh_set = sanitize_nonmanifoldness(pymesh_set)
       
        return pymesh_set, r
    
def resample_shape_pymeshlab(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
    pymesh = vedo.utils.vedo2meshlab(vedo_mesh)
    pymesh_set = pymeshlab.MeshSet()
    pymesh_set.add_mesh(pymesh)

    if pymesh_set.current_mesh().vertex_number() > threshold:
        ### !!!! we dont need a manifold shape for decimation
        last_vertex_count = -1
        prev_meshset = pymeshlab.MeshSet()
        prev_meshset.add_mesh(pymesh_set.current_mesh())
        pymesh_set.apply_filter("set_selection_all")
        current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
        pymesh_set.set_selection_none()
        while pymesh_set.current_mesh().vertex_number() > threshold:
            pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.9, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
            pymesh_set.apply_filter("set_selection_all")
            modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
            pymesh_set.set_selection_none()
            # print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
            
            if last_vertex_count == pymesh_set.current_mesh().vertex_number():
                break
            last_vertex_count = pymesh_set.current_mesh().vertex_number()

            # print(last_vertex_count)
        vedo_mesh, r = sanitize_mesh(pymesh_set)

        return MeshObject(pymesh_set.current_mesh(), visualize=True), r
    vedo_mesh, r = sanitize_mesh(pymesh_set)

    # print(pymesh_set.current_mesh().vertex_number())

    subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_ls3_loop", loopweight=2, iterations=1)
    if pymesh_set.current_mesh().vertex_number() < 100: # low vertex shapes' topology is heavily modified by any loop-adjacent subdivision algo
        subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_midpoint", iterations=1)
    
    last_vertex_count = -1
    while pymesh_set.current_mesh().vertex_number() < threshold:
        # print("!")
        subdiv_method()
        if last_vertex_count == pymesh_set.current_mesh().vertex_number():
            break
        last_vertex_count = pymesh_set.current_mesh().vertex_number()

    last_vertex_count = -1
    prev_meshset = pymeshlab.MeshSet()
    prev_meshset.add_mesh(pymesh_set.current_mesh())
    pymesh_set.apply_filter("set_selection_all")
    current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
    pymesh_set.set_selection_none() # not do decimation after subdivision as it introduces non manifoldness?
    while pymesh_set.current_mesh().vertex_number() > threshold:
        # print("?")
        pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.99, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
        pymesh_set.apply_filter("set_selection_all")
        modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
        pymesh_set.set_selection_none()
        # print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
        if modified_vertex_num/current_vertex_num < 0.8:
            pymesh_set = prev_meshset
            break
        if last_vertex_count == pymesh_set.current_mesh().vertex_number():
            break
        last_vertex_count = pymesh_set.current_mesh().vertex_number()

    # print(pymesh_set.current_mesh().vertex_number(), "last!!")
    # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

    return MeshObject(pymesh_set.current_mesh(), visualize=True), r

df = pd.read_pickle("original_data_statistics.pickle")

shape_path_orig = "../ShapeDatabase_INFOMR_orig"
classes = [class_type for class_type in os.listdir(shape_path_orig) if os.path.isdir((os.path.join(shape_path_orig, class_type))) and class_type != ".git"]
oops = defaultdict(list)
ratios = defaultdict(list)
has_nan_inf = []
for class_type in classes:
    if class_type != "Apartment":
        continue
    aux_df = df[df["class"] == class_type]
    iterator = tqdm(zip(aux_df["class"], aux_df["name"]), total=len(aux_df[aux_df["class"] == class_type]), desc=f"{class_type} - {''}")
    for class_type, obj_name in iterator:
        # if obj_name != "m465.obj":
            # continue
        iterator.set_description(f"{class_type} - {obj_name}")
        path = os.path.join(shape_path_orig, class_type, obj_name)
        mesh = MeshObject(path, True, class_type=class_type, name=obj_name)
        try:
            # mesh.show()
            mesh, ratio = resample_shape_pymeshlab(mesh)
            if np.sum(np.isnan(mesh.coordinates)):
                for c in mesh.coordinates:
                    print(c)
                mesh.show()
                break
            has_nan_inf += [[np.sum(np.isnan(mesh.coordinates)), np.sum(np.isinf(mesh.coordinates))]]
            # mesh.show()
        except:
            oops[class_type] += [path]
        ratios[class_type] += [ratio]

Apartment - D00086.obj:   0%|          | 0/23 [00:00<?, ?it/s]

Apartment - m465.obj:  22%|██▏       | 5/23 [00:06<00:23,  1.30s/it]  

[0.1323966  0.92397594 0.21973178]
[0.13055521 0.926727   0.22143704]
[0.1304597  0.9239757  0.22992548]
[0.13229527 0.926727   0.22822355]
[0.13225351 0.9082204  0.22149436]
[0.13027571 0.9054163  0.22025596]
[0.13060042 0.90822047 0.22816382]
[0.13257298 0.90541637 0.2294039 ]
[0.11356041 0.92523426 0.21082796]
[0.11442728 0.9227795  0.21252348]
[0.11951159 0.92523426 0.21271904]
[0.11864471 0.9227795  0.21102352]
[0.11403073 0.906527   0.21092196]
[0.11323049 0.9064762  0.21272892]
[0.11904126 0.906527   0.21262504]
[0.11984152 0.9064762  0.21081808]
[0.09421341 0.9254401  0.21082796]
[0.09508029 0.92298526 0.21252348]
[0.10016459 0.9254401  0.21271904]
[0.09929772 0.92298526 0.21102352]
[0.09468374 0.90673214 0.21092196]
[0.09388348 0.9066814  0.21272893]
[0.09969427 0.90673214 0.21262504]
[0.10049451 0.9066814  0.21081807]
[0.15465751 0.67882365 0.01404806]
[0.15796301 0.81150246 0.02099294]
[0.19391775 0.6798921  0.03175194]
[0.17552969 0.8105694  0.03599627]
[0.19824113 0.679546

Apartment - m465.obj:  22%|██▏       | 5/23 [00:09<00:34,  1.90s/it]


In [22]:
np.sum(np.asarray(has_nan_inf), axis=0)

array([0, 0])

In [ ]:
def sanitize_nonmanifoldness(pymesh_set):
    operations_nonmanifoldness = [
        partial(pymesh_set.meshing_repair_non_manifold_edges, method=0),
        # pymesh_set.compute_selection_by_non_manifold_edges_per_face,
        # pymesh_set.meshing_remove_selected_faces,
        pymesh_set.meshing_repair_non_manifold_vertices,
    ]

    is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
    while not is_manifold:
        for operation in operations_nonmanifoldness:
            operation()
            # print("?")
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        # print("done for loop")

    # print("done manifold")
    return pymesh_set

def sanitize_geometry(pymesh_set):
    def remove_disconnected_components(pymesh_set, nbfaceratio=0.05, ratio_selected_faces = 0.24): # elbow method?
        pymesh_set.apply_filter("set_selection_all")
        all_faces_num = pymesh_set.current_mesh().selected_face_number()
        pymesh_set.set_selection_none()
        pymesh_set.compute_selection_by_small_disconnected_components_per_face(nbfaceratio=nbfaceratio)
        r = pymesh_set.current_mesh().selected_face_number()/all_faces_num
        if r > ratio_selected_faces: # do it per class?
            # print("removed components")
            pymesh_set.meshing_remove_selected_faces()
        pymesh_set.set_selection_none()

        return pymesh_set, r

    operations_cleaning = [
        pymesh_set.meshing_remove_duplicate_faces,
        partial(remove_disconnected_components, pymesh_set, nbfaceratio=0.05),
        pymesh_set.meshing_merge_close_vertices,
    ]

    r = None
    for operation in operations_cleaning:
        if isinstance(operation, partial):
            _, r = operation()
        else:
            # print(operation.__name__)
            operation()

    return pymesh_set, r

def sanitize_mesh(pymesh_set):       
        pymesh_set, r = sanitize_geometry(pymesh_set)
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        if not is_manifold:
            pymesh_set = sanitize_nonmanifoldness(pymesh_set)

        # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])
        # print("pre_hole")

        pymesh_set.meshing_close_holes(refinehole=True, refineholeedgelen=pymeshlab.PercentageValue(1), maxholesize=75)
        # print("hole")
        pymesh_set.set_selection_none()
        is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
        if not is_manifold:
            pymesh_set = sanitize_nonmanifoldness(pymesh_set)
       
        return pymesh_set, r
    
def resample_shape_pymeshlab(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
    pymesh = vedo.utils.vedo2meshlab(vedo_mesh)
    pymesh_set = pymeshlab.MeshSet()
    pymesh_set.add_mesh(pymesh)

    if pymesh_set.current_mesh().vertex_number() > threshold:
        ### !!!! we dont need a manifold shape for decimation
        last_vertex_count = -1
        prev_meshset = pymeshlab.MeshSet()
        prev_meshset.add_mesh(pymesh_set.current_mesh())
        pymesh_set.apply_filter("set_selection_all")
        current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
        pymesh_set.set_selection_none()
        while pymesh_set.current_mesh().vertex_number() > threshold:
            pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.9, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
            pymesh_set.apply_filter("set_selection_all")
            modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
            pymesh_set.set_selection_none()
            # print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
            
            if last_vertex_count == pymesh_set.current_mesh().vertex_number():
                break
            last_vertex_count = pymesh_set.current_mesh().vertex_number()

            # print(last_vertex_count)
        vedo_mesh, r = sanitize_mesh(pymesh_set)

        return MeshObject(pymesh_set.current_mesh(), visualize=True), r
    vedo_mesh, r = sanitize_mesh(pymesh_set)

    # print(pymesh_set.current_mesh().vertex_number())

    subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_ls3_loop", loopweight=2, iterations=1)
    if pymesh_set.current_mesh().vertex_number() < 100: # low vertex shapes' topology is heavily modified by any loop-adjacent subdivision algo
        subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_midpoint", iterations=1)
    
    last_vertex_count = -1
    while pymesh_set.current_mesh().vertex_number() < threshold:
        # print("!")
        subdiv_method()
        if last_vertex_count == pymesh_set.current_mesh().vertex_number():
            break
        last_vertex_count = pymesh_set.current_mesh().vertex_number()

    last_vertex_count = -1
    prev_meshset = pymeshlab.MeshSet()
    prev_meshset.add_mesh(pymesh_set.current_mesh())
    pymesh_set.apply_filter("set_selection_all")
    current_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
    pymesh_set.set_selection_none() # not do decimation after subdivision as it introduces non manifoldness?
    while pymesh_set.current_mesh().vertex_number() > threshold:
        # print("?")
        pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.99, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
        pymesh_set.apply_filter("set_selection_all")
        modified_vertex_num = pymesh_set.current_mesh().selected_vertex_number()
        pymesh_set.set_selection_none()
        # print(modified_vertex_num/current_vertex_num, modified_vertex_num, current_vertex_num)
        if modified_vertex_num/current_vertex_num < 0.8:
            pymesh_set = prev_meshset
            break
        if last_vertex_count == pymesh_set.current_mesh().vertex_number():
            break
        last_vertex_count = pymesh_set.current_mesh().vertex_number()

    # print(pymesh_set.current_mesh().vertex_number(), "last!!")
    # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

    return MeshObject(pymesh_set.current_mesh(), visualize=True), r

df = pd.read_pickle("original_data_statistics.pickle")

shape_path_orig = "../ShapeDatabase_INFOMR_orig"
classes = [class_type for class_type in os.listdir(shape_path_orig) if os.path.isdir((os.path.join(shape_path_orig, class_type))) and class_type != ".git"]
oops = defaultdict(list)
ratios = defaultdict(list)
for class_type in classes:
    # if class_type != "Chess":
    #     continue
    aux_df = df[df["class"] == class_type]
    iterator = tqdm(zip(aux_df["class"], aux_df["name"].iloc[:]), total=len(aux_df[aux_df["class"] == class_type]), desc=f"{class_type} - {''}")
    for class_type, obj_name in iterator:
        # if obj_name != "m1595.obj":
        #     continue
        iterator.set_description(f"{class_type} - {obj_name}")
        path = os.path.join(shape_path_orig, class_type, obj_name)
        mesh = MeshObject(path, True, class_type=class_type, name=obj_name)
        try:
            mesh, ratio = resample_shape_pymeshlab(mesh)
        except:
            oops[class_type] += [path]
        del mesh
        ratios[class_type] += [ratio]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
all_r = []
mean_r = []
for class_type, r in list(ratios.items())[:]:
    mean_r += [np.mean(r)]
    r = np.sort(r)[::-1][:15]
    all_r += [r]
    sns.lineplot(r)

all_r = np.asarray(all_r)
print(np.mean(all_r))
print(np.mean(mean_r))
# plt.show()

In [ ]:
# def sanitize_nonmanifoldness(pymesh_set):
#     operations_nonmanifoldness = [
#         partial(pymesh_set.meshing_repair_non_manifold_edges, method=1),
#         pymesh_set.compute_selection_by_non_manifold_edges_per_face,
#         pymesh_set.meshing_remove_selected_faces,
#         pymesh_set.meshing_repair_non_manifold_vertices,
#     ]

#     is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#     while not is_manifold:
#         for operation in operations_nonmanifoldness:
#             operation()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]

#     return pymesh_set

# def sanitize_geometry(pymesh_set, do_subdivision=True):
#     def remove_disconnected_components(pymesh_set, nbfaceratio=0.25, ratio_selected_faces = 0.24, do_subdivision=True): # elbow method?
#         pymesh_set.apply_filter("set_selection_all")
#         all_faces_num = pymesh_set.current_mesh().selected_face_number()
#         pymesh_set.set_selection_none()
#         pymesh_set.compute_selection_by_small_disconnected_components_per_face(nbfaceratio=nbfaceratio)
#         r = pymesh_set.current_mesh().selected_face_number()/all_faces_num
#         if r > ratio_selected_faces and do_subdivision: # do it per class?
#             # print("removed components")
#             pymesh_set.meshing_remove_selected_faces()
#         pymesh_set.set_selection_none()

#         return pymesh_set, r

#     operations_cleaning = [
#         pymesh_set.meshing_remove_duplicate_faces,
#         partial(remove_disconnected_components, pymesh_set, nbfaceratio=0.25, do_subdivision=do_subdivision),
#         pymesh_set.meshing_merge_close_vertices,
#     ]

#     r = None
#     for operation in operations_cleaning:
#         if isinstance(operation, partial):
#             _, r = operation()
#         else:
#             operation()

#     return pymesh_set, r

# def sanitize_mesh(pymesh_set, do_subdivision=True):       
#         pymesh_set, r = sanitize_geometry(pymesh_set, do_subdivision=do_subdivision)
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)

#         # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])
#         # print("pre_hole")

#         pymesh_set.meshing_close_holes(refinehole=True, refineholeedgelen=pymeshlab.PercentageValue(1), maxholesize=75)
#         # print("hole")
#         pymesh_set.set_selection_none()
#         is_manifold = pymesh_set.get_topological_measures()["is_mesh_two_manifold"]
#         if not is_manifold:
#             pymesh_set = sanitize_nonmanifoldness(pymesh_set)
       
#         return pymesh_set, r

# def subdivide_mesh(pymesh_set, threshold):
#     subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_ls3_loop", loopweight=2, iterations=1)
#     if pymesh_set.current_mesh().vertex_number() < 100: # low vertex shapes' topology is heavily modified by any loop-adjacent subdivision algo
#         subdiv_method = partial(pymesh_set.apply_filter, filter_name="meshing_surface_subdivision_midpoint", iterations=1)
    
#     last_vertex_count = -1
#     while pymesh_set.current_mesh().vertex_number() < threshold:
#         # print("!")
#         subdiv_method()
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#     return pymesh_set

# def decimate_mesh(pymesh_set, threshold):
#     ### !!!! we dont need a manifold shape for decimation
#     last_vertex_count = -1
#     while pymesh_set.current_mesh().vertex_number() > threshold:
#         pymesh_set.apply_filter("meshing_decimation_quadric_edge_collapse", targetperc=0.99, qualitythr=0.3, preserveboundary=True, preservetopology=True, planarquadric=True)
        
#         if last_vertex_count == pymesh_set.current_mesh().vertex_number():
#             break
#         last_vertex_count = pymesh_set.current_mesh().vertex_number()

#             # print(last_vertex_count)

#     return pymesh_set
    
# def resample_shape_pymeshlab(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
#     pymesh = vedo.utils.vedo2meshlab(vedo_mesh)
#     pymesh_set = pymeshlab.MeshSet()
#     pymesh_set.add_mesh(pymesh)

#     # decimation first if object is big
#     if pymesh_set.current_mesh().vertex_number() > threshold:
#         pymesh_set = decimate_mesh(pymesh_set, threshold)
#         pymesh_set, r = sanitize_mesh(pymesh_set, do_subdivision=False)
#     else: # subdivide then decimate
#         pymesh_set, r = sanitize_mesh(pymesh_set, do_subdivision=True)
#         pymesh_set = subdivide_mesh(pymesh_set, threshold)
#         pymesh_set = decimate_mesh(pymesh_set, threshold)

#     # print(pymesh_set.current_mesh().vertex_number(), "last!!")
#     # print(pymesh_set.get_topological_measures()["is_mesh_two_manifold"])

#     return MeshObject(pymesh_set.current_mesh(), visualize=True), r

# df = pd.read_pickle("original_data_statistics.pickle")

# shape_path_orig = "../ShapeDatabase_INFOMR_orig"
# classes = [class_type for class_type in os.listdir(shape_path_orig) if os.path.isdir((os.path.join(shape_path_orig, class_type))) and class_type != ".git"]
# oops = defaultdict(list)
# ratios = defaultdict(list)
# for class_type in classes:
#     # if class_type != "Chess":
#     #     continue
#     aux_df = df[df["class"] == class_type]
#     iterator = tqdm(zip(aux_df["class"], aux_df["name"].iloc[:]), total=len(aux_df[aux_df["class"] == class_type]), desc=f"{class_type} - {''}")
#     for class_type, obj_name in iterator:
#         # if obj_name != "m1595.obj":
#         #     continue
#         iterator.set_description(f"{class_type} - {obj_name}")
#         path = os.path.join(shape_path_orig, class_type, obj_name)
#         mesh = MeshObject(path, True, class_type=class_type, name=obj_name)
#         try:
#             mesh, ratio = resample_shape_pymeshlab(mesh)
#         except:
#             oops[class_type] += [path]
#         del mesh
#         ratios[class_type] += [ratio]

In [ ]:
ratios

In [ ]:
def resample_shape(vedo_mesh: MeshObject, sampling_type: str="butterfly", threshold: int=5610, decimation_type = "simple"):
    vedo_mesh = sanitize_mesh(vedo_mesh)
    # add sanitation for ALL meshes (and restrict non manifoldness stuff)

    match sampling_type:
        case "loop":
            sampling_type = 0
        case "linear":
            sampling_type = 1
        case "adaptive":
            sampling_type = 2
        case "butterfly":
            sampling_type = 3
        case "centroid":
            sampling_type = 4

    last_vertex_count = -1
    while vedo_mesh.n_vertices < threshold:
        print(last_vertex_count)
        vedo_mesh = vedo_mesh.subdivide(1, method=sampling_type)
        if last_vertex_count == vedo_mesh.n_vertices:
            break
        last_vertex_count = vedo_mesh.n_vertices
    # vedo_mesh.decimate(n=threshold)
    # match decimation_type:
    #     case "simple":
    #         vedo_mesh.decimate(fraction=0.5, n=threshold)
    #     case "pro":
    #         vedo_mesh.decimate_pro(fraction=0.5, n=threshold)

    return vedo_mesh